# Hightran

In [ ]:

import numpy as np
import math
import time
def read_par_file(file_path):
    """
    Reads a PAR file and extracts parameters as a dictionary.
    Handles sections, comments, and key-value pairs.
    
    
    :param file_path: Path to the PAR file.
    :return: Dictionary of parameters with section headers as keys (if present).
    """
    parameters = {}
    current_section = None
    
    with open(file_path, 'r') as file:
        for line in file:
            # Strip comments and whitespace
            line = line.split('#')[0].strip()
            line = line.split('//')[0].strip()  # Handle // comments
            
            # Skip empty lines
            if not line:
                continue
            
            # Handle sections if present (e.g., [Section])
            if line.startswith('[') and line.endswith(']'):
                current_section = line[1:-1].strip()
                if current_section not in parameters:
                    parameters[current_section] = {}
                continue
            
            # Handle key-value pairs
            if '=' in line:
                key, value = map(str.strip, line.split('=', 1))
                
                # Convert value to appropriate data type (e.g., float, int, bool)
                if value.lower() in ('true', 'false'):
                    value = value.lower() == 'true'
                else:
                    try:
                        value = float(value) if '.' in value else int(value)
                    except ValueError:
                        value = value.strip('"')  # Remove quotes for strings
                
                # Add to current section or global parameters
                if current_section:
                    parameters[current_section][key] = value
                else:
                    parameters[key] = value
    
    return parameters
MAXLINES = 100000
NVALUES = 5
PI = 3.1415926535897932385

# Constants
KAPPALO = 500.0
KAPPAHI = 750.0
DELTAKAPPA = 0.0603 / 2

# Initialize data structure to store VAL
VAL = np.zeros((MAXLINES, NVALUES))

# Read HiTran data from the input file
input_file = r'Hitran.par'
output_file = r'CO2abs.csv'

nl = 0

try:
    params = read_par_file(input_file)
    print(params)
    with open(input_file, 'r') as file:
        for line in file:
            # Formatting in Python is different, we read from the file directly
            if nl >= MAXLINES:
                break
            elements = line.split()
            if len(elements) < 7:
                continue
            ia = int(elements[0])  # Unused
            ib = int(elements[1])  # Unused
            for j in range(NVALUES):
                VAL[nl, j] = float(elements[j + 2])
            nl += 1
except FileNotFoundError:
    print(f"File {input_file} not found.")
    exit(1)

NLINES = nl

# Function to calculate the indefinite integral of Lorentz line shape function
def ILNZ(k, k0, g):
    return math.atan2((k - k0), g) / PI

# Perform integration and write to the output file
start_time = time.time()

with open(output_file, 'w') as file:
    for kappa in np.arange(KAPPALO + DELTAKAPPA, KAPPAHI + DELTAKAPPA, 2 * DELTAKAPPA):
        S = 0.0
        for line in range(NLINES):
            v1 = ILNZ(kappa + DELTAKAPPA, VAL[line, 0], VAL[line, 3])
            v2 = ILNZ(kappa - DELTAKAPPA, VAL[line, 0], VAL[line, 3])
            V = v1 - v2
            S += VAL[line, 1] * V
        S = S / (2 * DELTAKAPPA)
        file.write(f"{kappa:.6E},{S:.6E}\n")
        # Optional console output
        print(f"{kappa:.6E},{S:.6E}")

end_time = time.time()

# Optional statistics output
print(f"{NLINES} Lines, {(KAPPAHI - KAPPALO) / 2 / DELTAKAPPA} Values, {end_time - start_time} seconds")






# Clive Best Code

In [ ]:
import math
import numpy as np

# Define constants
sigma = 0.0000000567
frac = 0.2  # Fraction of IR in the CO2 absorption band
TS = 288  # Surface temperature in Kelvin
lapse = 0.0065  # Lapse rate in K/m
CO2 = 0.0003  # CO2 concentration
k = 1.48  # Beer-Lambert parameter for CO2 absorption band
scale = 8600.0  # Scale height in meters
a = 100.0  # Slice thickness in meters

# Emission from the ground
rtup = frac * sigma * TS**4
radup = [rtup]
atmrad = 0.0

# Divide atmosphere into 100m slices
for i in range(151):
    z = i * 100.0
    
    # Temperature at level i
    if i < 111:
        T = TS - z * lapse
    else:
        T = TS - 111 * 100 * lapse  # Constant temperature above 11 km
    
    # Calculate the partial pressure of CO2 at level i
    P = CO2 * math.exp(-z / scale)
    
    # Transmission and absorption using Beer-Lambert law
    trans = math.exp(-k * P * a)
    absorp = 1 - trans
    
    # Using Kirchoff's law - equal flux emitted by level i to that absorbed
    rtup = radup[i - 1] * trans + absorp * frac * sigma * T**4
    
    # Now subtract the downward radiation from all the levels above i
    rdown = 0.0
    for j in range(i + 2, 153):
        zz = j * 100.0
        P = CO2 * math.exp(-zz / scale)
        local = math.exp(-k * P * a)
        absorp = 1 - local
        TU = TS - zz * lapse if j < 111 else TS - 111 * 100 * lapse
        remit = absorp * sigma * frac * TU**4
        
        # Integrate down through the layers
        rleft = remit
        for m in range(j, i + 1, -1):
            zk = m * 100.0
            Pk = CO2 * math.exp(-zk / scale)
            trans = math.exp(-k * Pk * a)
            rleft *= trans
        rdown += rleft
    
    # Update radup for the next layer
    radup.append(rtup)
    rtup -= rdown
    
    atmrad = rtup - radup[0]
    
    # Print the results for each layer
    print(f"{i}    {rtup:.6f}    {rdown:.6f}")


0    20.849470    57.166409
1    21.537446    56.448294
2    22.221833    55.734651
3    22.873702    55.025516
4    23.522690    54.320926
5    24.141036    53.620917
6    24.757119    52.925524
7    25.344277    52.234782
8    25.929709    51.548725
9    26.487787    50.867387
10    27.044608    50.190801
11    27.575516    49.518999
12    28.105574    48.852012
13    28.611039    48.189871
14    29.116009    47.532607
15    29.597600    46.880249
16    30.079000    46.232825
17    30.538140    45.590362
18    30.997352    44.952888
19    31.435333    44.320428
20    31.873613    43.693008
21    32.291612    43.070651
22    32.710103    42.453381
23    33.109190    41.841218
24    33.508936    41.234186
25    33.890091    40.632304
26    34.272043    40.035590
27    34.636158    39.444064
28    35.001189    38.857741
29    35.349082    38.276639
30    35.697991    37.700771
31    36.030411    37.130152
32    36.363931    36.564795
33    36.681567    36.004710
34    37.000373    35.44